In [3]:
import sys
import os
import time
import snap
print snap.Version
import testutils

context = snap.TTableContext()

t = testutils.Timer()
r = testutils.Resource()

# load tables and context
srcfile = "/dfs/scratch0/viswa/fromlfs/mag-022016-papers-paa-authors-affls-refs-context-net.bin"
FIn = snap.TFIn(srcfile)

# load papers
print time.ctime(), "loading papers ..."
papers = snap.TTable.Load(FIn,context)
t.show("loadbin papers", papers)
r.show("__papers__")

print time.ctime(), "loading paa ..."
paa = snap.TTable.Load(FIn,context)
t.show("loadbin paa", paa)
r.show("__paa__")

print time.ctime(), "loading auth ..."
auth = snap.TTable.Load(FIn,context)
t.show("loadbin auth", auth)
r.show("__auth__")

print time.ctime(), "loading affl ..."
affl = snap.TTable.Load(FIn,context)
t.show("loadbin affl", affl)
r.show("__affl__")

# load references
print time.ctime(), "loading references ..."
refs = snap.TTable.Load(FIn,context)
t.show("loadbin references", refs)
r.show("__references__")

# load context
print time.ctime(), "loading context ..."
context.Load(FIn)
t.show("loadbin context", refs)
r.show("__context__")

# load network
#print time.ctime(), "loading network ..."
#net = snap.TNEANet.Load(FIn)
#t.show("loadbin network", net)
#r.show("__network__")

print time.ctime(), "done"


2.0.3-dev
Thu Apr 14 10:08:34 2016 loading papers ...
[loadbin papers]  Elapsed: 32.42 seconds, Rows: 126909021
__papers__      	cpu(s) 32.337	mem(MB) 90869.168
Thu Apr 14 10:09:07 2016 loading paa ...
[loadbin paa]     Elapsed: 55.35 seconds, Rows: 337000600
__paa__         	cpu(s) 55.177	mem(MB) 90869.168
Thu Apr 14 10:10:02 2016 loading auth ...
[loadbin auth]    Elapsed: 9.60 seconds, Rows: 114698044
__auth__        	cpu(s) 9.579	mem(MB) 90869.168
Thu Apr 14 10:10:11 2016 loading affl ...
[loadbin affl]    Elapsed: 0.00 seconds, Rows: 19843
__affl__        	cpu(s) 0.002	mem(MB) 90869.168
Thu Apr 14 10:10:11 2016 loading references ...
[loadbin references]Elapsed: 44.19 seconds, Rows: 528682289
__references__  	cpu(s) 44.029	mem(MB) 90869.168
Thu Apr 14 10:10:56 2016 loading context ...
[loadbin context] Elapsed: 118.03 seconds, Rows: 528682289
__context__     	cpu(s) 117.748	mem(MB) 90869.168
Thu Apr 14 10:12:54 2016 done


In [7]:
schema = map(lambda x: x.GetVal1(), refs.GetSchema())
print time.ctime(), "Creating network ..."
net = snap.ToGraph(snap.PNGraph, refs, schema[0], schema[1], snap.aaFirst)
print time.ctime(), "done."

Thu Apr 14 10:20:01 2016 Creating network ...
Thu Apr 14 10:32:23 2016 done.


In [8]:
print time.ctime(), "computing indegv ..."
InDegV = snap.TIntPrV()
snap.GetNodeInDegV(net, InDegV)
t.show("indegv", InDegV)
r.show("__InDegV__")
print time.ctime()

Thu Apr 14 10:37:52 2016 computing indegv ...
[indegv]          Elapsed: 1499.50 seconds, Length: 46742304
__InDegV__      	cpu(s) 3750.534	mem(MB) 91295.560
Thu Apr 14 10:37:53 2016


In [9]:
print time.ctime(), "computing pagerank ..."
PRankH = snap.TIntFltH()
snap.GetPageRankMP(net, PRankH, 0.85, 1e-4, 100)
t.show("prank", PRankH)
r.show("__PRankH__")
print time.ctime()

Thu Apr 14 10:38:06 2016 computing pagerank ...
[prank]           Elapsed: 1110.43 seconds, Length: 46742304
__PRankH__      	cpu(s) 1095.239	mem(MB) 91340.572
Thu Apr 14 10:56:24 2016


In [10]:
print time.ctime(), "creating TTable with PageRank ..."
PRankT = snap.TTable(PRankH, "PaperID", "PageRankScore", context, snap.TBool(True))
t.show("prankt", PRankT)
r.show("__PRankT__")
print time.ctime()

Thu Apr 14 12:27:37 2016 creating TTable with PageRank ...
[prankt]          Elapsed: 5481.42 seconds
__PRankT__      	cpu(s) 8.162	mem(MB) 91340.572
Thu Apr 14 12:27:45 2016


In [11]:
print time.ctime(), "Creating table after join ..."
PaperPRankJoinT = papers.Join("PaperID", PRankT, "PaperID")
t.show("paperprankjoint", PaperPRankJoinT)
r.show("__PaperPRankJoinT__")
print time.ctime()

Thu Apr 14 12:27:54 2016 Creating table after join ...
[paperprankjoint] Elapsed: 170.70 seconds, Rows: 46742304
__PaperPRankJoinT__	cpu(s) 161.384	mem(MB) 94924.608
Thu Apr 14 12:30:36 2016


In [12]:
print time.ctime()
orderby = snap.TStrV()
orderby.Add("PageRankScore")
PaperPRankJoinT.Order(orderby, "", snap.TBool(False), snap.TBool(False))
print time.ctime()

Thu Apr 14 12:31:32 2016
Thu Apr 14 12:48:52 2016


In [13]:
print time.ctime()
InDegH = snap.TIntH()
i = 0
for item in InDegV:
    i += 1
    if i % 1000000 == 0:
        print i
    InDegH.AddDat(item.GetVal1(), item.GetVal2())
#InDegT = snap.TTable(PRankH, "PaperID", "PageRankScore", context, snap.TBool(True))
print time.ctime()

Thu Apr 14 12:48:55 2016
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
Thu Apr 14 12:52:58 2016


In [14]:
print time.ctime()
InDegT = snap.TTable(InDegH, "PaperID", "InDeg", context, snap.TBool(True))
print time.ctime()

Thu Apr 14 12:54:04 2016
Thu Apr 14 12:54:12 2016


In [15]:
print time.ctime(), "Creating table after join ..."
PaperInDegJoinT = papers.Join("PaperID", InDegT, "PaperID")
print time.ctime()

Thu Apr 14 12:54:22 2016 Creating table after join ...
Thu Apr 14 12:56:57 2016


In [16]:
print time.ctime()
orderby = snap.TStrV()
orderby.Add("InDeg")
PaperInDegJoinT.Order(orderby, "", snap.TBool(False), snap.TBool(False))
print time.ctime()

Thu Apr 14 12:57:02 2016
Thu Apr 14 12:57:38 2016


In [ ]:
"""
paa_rit = paa.BegRI()
schema = paa.GetSchema()
#for i in range(schema.Len()):
#    print schema[i].GetVal1(), schema[i].GetVal2()
i = 0
print dir(paa)
while paa_rit < paa.EndRI():
    print paa_rit.GetRowIdx(), paa_rit.GetStrMapByName("PaperID"), context.GetStr(paa_rit.GetStrMapByName("PaperID"))
    print paa.GetStrVal("PaperID", paa_rit.GetRowIdx())
    paa_rit.Next()
    print 'Next'
    i += 1
    if i > 4:
        break
"""

In [20]:
print time.ctime()
auth_schema = auth.GetSchema()
for i in range(auth_schema.Len()):
    print auth_schema[i].GetVal1(), auth_schema[i].GetVal2()
auth.RequestIndexStrMap("AuthorID")
print time.ctime()

Thu Apr 14 13:01:49 2016
AuthorID 2
AuthorName 2
_id 0
Thu Apr 14 13:03:40 2016


In [22]:
print time.ctime()
affl_schema = affl.GetSchema()
for i in range(affl_schema.Len()):
    print affl_schema[i].GetVal1(), affl_schema[i].GetVal2()
affl.RequestIndexStrMap("AfflID")
print time.ctime()

Thu Apr 14 13:04:01 2016
AfflID 2
AfflName 2
_id 0
Thu Apr 14 13:04:01 2016


In [23]:
print time.ctime()
PaperAAPRJoinT = PaperPRankJoinT.Join("PaperID", paa, "PaperID")
print time.ctime()

Thu Apr 14 13:12:39 2016
Thu Apr 14 13:21:47 2016


In [26]:
paperaapr_it = PaperAAPRJoinT.BegRI()
i = 0
while paperaapr_it < PaperAAPRJoinT.EndRI():
    paperaapr_it.Next()
    i += 1
    if i % 10000000 == 0:
        print i
print i

10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
100000000
110000000
120000000
130000000
140000000
150000000
160000000
160022074


In [27]:
paperpr_it = PaperPRankJoinT.BegRI()
i = 0
while paperpr_it < PaperPRankJoinT.EndRI():
    paperpr_it.Next()
    i += 1
    if i % 10000000 == 0:
        print i
print i

10000000
20000000
30000000
40000000
46742304


In [33]:
paperaapr_it = PaperAAPRJoinT.BegRI()
i = 0
s = PaperAAPRJoinT.GetSchema()
#for i in range(s.Len()):
#   print s[i].GetVal1(), s[i].GetVal2()
i = 0
auth_to_score = snap.TIntFltH()
while paperaapr_it < PaperAAPRJoinT.EndRI():
    auth_id = paperaapr_it.GetStrMapByName("AuthorID")
    score = paperaapr_it.GetFltAttr("PageRankScore")
    if auth_to_score.IsKey(auth_id):
        old_score = auth_to_score.GetDat(auth_id)
        new_score = old_score + score
        auth_to_score.AddDat(auth_id, new_score)
    else:
        auth_to_score.AddDat(auth_id, score)
    paperaapr_it.Next()
    i += 1
    if i % 10000000 == 0:
        print i
print i

10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
100000000
110000000
120000000
130000000
140000000
150000000
160000000
160022074


In [34]:
print time.ctime()
AuthorPRankT = snap.TTable(auth_to_score, "AuthorID", "PageRankScore", context, snap.TBool(True))
print time.ctime()

Thu Apr 14 14:06:13 2016
Thu Apr 14 14:06:20 2016


In [35]:
print time.ctime()
orderby = snap.TStrV()
orderby.Add("PageRankScore" )
AuthorPRankT.Order(orderby, "", snap.TBool(False), snap.TBool(False))
print time.ctime()

Thu Apr 14 14:07:30 2016
Thu Apr 14 14:16:34 2016


In [38]:
apr_it = AuthorPRankT.BegRI()
i = 0
while apr_it < AuthorPRankT.EndRI():
    
    print apr_it.GetIntAttr("AuthorID"), context.GetStr(apr_it.GetIntAttr("AuthorID")), apr_it.GetFltAttr("PageRankScore")
    
    apr_it.Next()
    i += 1
    if i > 5:
        break

24591336 7A1262EF 0.000373207117574
30975895 10.1007/BF02525835 0.000372893499269
29907397 The Histologic Characteristics of the Periapical Tissue with Regard to Root-Canal Treatment * 0.000370402487903
32319422 x ray diffraction and vibrational spectroscopic studies of indolecarboxylic acids and their metal complexes 0.000370402487903
22035 cocyclic jacket matrices and its application to cryptography systems 0.000172062609994
2035020 10.1109/VETECF.2008.464 0.000135701122354


In [39]:
print time.ctime()
PaperAAInDegJoinT = PaperInDegJoinT.Join("PaperID", paa, "PaperID")
print time.ctime()

paperaaid_it = PaperAAInDegJoinT.BegRI()
i = 0
s = PaperAAInDegJoinT.GetSchema()
#for i in range(s.Len()):
#   print s[i].GetVal1(), s[i].GetVal2()


Thu Apr 14 15:09:24 2016
Thu Apr 14 15:17:37 2016


RuntimeError: Execution stopped: (0<=ValN)&&(ValN<Vals) [Reason:'Index:-1 Vals:7 MxVals:7 Type:4TVecIS_I4TIntiEiE'], file ../../snap-dev/glib-core/ds.h, line 463

In [40]:
i = 0
auth_to_deg = snap.TIntH()
print time.ctime()
while paperaaid_it < PaperAAInDegJoinT.EndRI():
    auth_id = paperaaid_it.GetStrMapByName("AuthorID")
    deg = paperaaid_it.GetIntAttr("InDeg")
    if auth_to_deg.IsKey(auth_id):
        old_deg = auth_to_deg.GetDat(auth_id)
        new_deg = old_deg + deg
        auth_to_deg.AddDat(auth_id, new_deg)
    else:
        auth_to_deg.AddDat(auth_id, deg)
    paperaaid_it.Next()
    i += 1
    if i % 10000000 == 0:
        print i
print i
print time.ctime()

Thu Apr 14 15:19:25 2016
10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
100000000
110000000
120000000
130000000
140000000
150000000
160000000
160022074
Thu Apr 14 15:38:39 2016


In [41]:
print time.ctime()
AuthorInDegT = snap.TTable(auth_to_deg, "AuthorID", "InDeg", context, snap.TBool(True))
print time.ctime()
print time.ctime()
orderby = snap.TStrV()
orderby.Add("InDeg")
AuthorInDegT.Order(orderby, "", snap.TBool(False), snap.TBool(False))
print time.ctime()

Thu Apr 14 15:41:01 2016
Thu Apr 14 15:41:09 2016
Thu Apr 14 15:41:09 2016
Thu Apr 14 15:41:22 2016


In [ ]:
i = 0
affl_to_deg = snap.TIntH()
print time.ctime()
paperaaid_it = PaperAAInDegJoinT.BegRI()
while paperaaid_it < PaperAAInDegJoinT.EndRI():
    affl_id = paperaaid_it.GetStrMapByName("AfflID")
    deg = paperaaid_it.GetIntAttr("InDeg")
    if affl_to_deg.IsKey(affl_id):
        old_deg = affl_to_deg.GetDat(affl_id)
        new_deg = old_deg + deg
        affl_to_deg.AddDat(affl_id, new_deg)
    else:
        affl_to_deg.AddDat(affl_id, deg)
    paperaaid_it.Next()
    i += 1
    if i % 10000000 == 0:
        print i
print i
print time.ctime()

Thu Apr 14 15:58:29 2016
10000000

In [47]:
print time.ctime()
AfflInDegT = snap.TTable(affl_to_deg, "AfflID", "InDeg", context, snap.TBool(True))
print time.ctime()
print time.ctime()
orderby = snap.TStrV()
orderby.Add("InDeg")
AfflInDegT.Order(orderby, "", snap.TBool(False), snap.TBool(False))
print time.ctime()

Thu Apr 14 15:56:48 2016


RuntimeError: Execution stopped: (0<=ValN)&&(ValN<Vals) [Reason:'Index:-1 Vals:0 MxVals:0 Type:4TVecI4TIntiE'], file ../../snap-dev/glib-core/ds.h, line 467

In [17]:
r_it = PaperPRankJoinT.BegRI()
schema = PaperPRankJoinT.GetSchema()
num_cols = schema.Len()
for i in range(num_cols):
    print schema[i].GetVal1(), schema[i].GetVal2()
for i in range(10):
    print r_it.GetRowIdx(), context.GetStr(r_it.GetStrMapByName("PaperTitle")), r_it.GetFltAttr("PageRankScore"),
    r_it.Next()
    i += 1
    if i > 4:
        break


PaperID-1 2
PaperTitle 2
PaperTitleNorm 2
PaperYear 2
PaperDate 2
PaperDOI 2
VenueName 2
VenueNameNorm 2
JournalID 2
SeriesID 2
PaperRank 0
_id-1 0
PaperID-2 2
PageRankScore 1
_id-2 0
_id 0
5221350 Protein measurement with the folin phenol reagent 0.000370088869598 1832980 Cleavage of Structural Proteins during the Assembly of the Head of Bacteriophage T4 0.000125625544882 7441325 A rapid and sensitive method for the quantitation of microgram quantities of protein utilizing the principle of protein-dye binding. 9.73135715145e-05 1145686 Molecular cloning : a laboratory manual 6.29855443385e-05 14003718 The nature of the chemical bond 4.77545138718e-05
